In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, text
#create_engine - used to connect databases
#Column - used to define table columns inside classes
#Table - Class
#Column-Attribute, Row - Object
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
#orm-Object Relational Mapping.
engine = create_engine("sqlite:///fintrack.db")
Base = declarative_base()
#all tables will inherit from base class
#SQLAlchemy doesn’t know which classes are database tables.
Session = sessionmaker(bind=engine)
session = Session() 
#Session = manager for database operations.
class Category(Base):
    __tablename__ = "categories"
    id = Column(Integer, primary_key = True)
    name = Column(String)
    expenses = relationship("Expense", back_populates = "category")
class Expense(Base):
    __tablename__ = "expenses"
    id = Column(Integer, primary_key = True)
    title = Column(String)
    amount = Column(Float)
    date = Column(String)
    #we used ForeignKey here because we will category as food only one expenses, but in food items so many will be there
    category_id = Column(Integer, ForeignKey("categories.id"))
    category = relationship("Category", back_populates = "expenses")
class Subscription(Base):
    __tablename__ = "subscriptions"
    id = Column(Integer, primary_key = True)
    name = Column(String)
    amount = Column(Float)
    next_date = Column(String)
class Budget(Base):
    __tablename__ = "budgets"
    id = Column(Integer, primary_key = True)
    month = Column(String)
    limit = Column(Float)
#metadata-Information about all tables.”
Base.metadata.create_all(engine)
def add_category():
    name = input("Category Name: ")
    session.add(Category(name=name))
    session.commit()
    print("Category done")
def add_expense():
    title = input("Expense title: ")
    amount = float(input("Amount is: "))
    date = input("Date (YYYY-MM-DD): ")
    category_id = int(input("Category ID: "))
    session.add(Expense(title=title, amount = amount, date = date, category_id = category_id))
    session.commit()
    print("Expense Done: ")
def update_expense():
    e_id = int(input("Expense ID:"))
    expense = session.query(Expense).filter(Expense.id == e_id).first()
    if expense:
        expense.title = input("New title: ")
        expense.amount = float(input("New amount"))
        session.commit()
        print("Expense updated Successfully")
    else:
        print("Expense did not found")
def delete_expense():
    eid = int(input("Expense ID: "))
    expense = session.query(Expense).filter(Expense.id == eid).first()
    if expense:
        session.delete(expense)
        session.commit()
        print("Expense deleted successfully")
    else:
        print("Expense not founded")

def search_by_date():
    date = input("Enter date (YYYY-MM-DD): ")
    expenses = session.query(Expense).filter(Expense.date == date).all()
    print("Expenses on", date)
    for e in expenses:
        print(e.title,"-",e.amount,"-",e.category.name)

def category_report():
    sql ="""
    select c.name, sum(e.amount)
    from categories c 
    join expenses e on c.id = e.category_id
    group by c.name
    """
    result = session.execute(text(sql))
    print("Spending details: ")
    for ia in result:
        print(ia[0], "Rs" , ia[1])
        
def budget_alert():#Function to alert if budget exceeded
    month = input("Enter month to check: ")
    budget = session.query(Budget).filter_by(month=month).first()#Query budget for the given month
    if not budget:#If no budget set for the month
        print(" No budget set for this month")
        return
    sql1 = """
        SELECT SUM(amount) FROM expenses
    """
    total = session.execute(text(sql1)).scalar()
    print("Total Spending:",total)
    print("Budget Limit:",budget.limit)
    

    if total > budget.limit:
        print(" Budget Exceeded!")
    else:
        print(" Within Budget")

def set_budget():
    month = input("Month (YYYY-MM): ")
    limit = float(input("Monthly limit: "))
    session.add(Budget(month=month, limit=limit))
    session.commit()
    print("Budget set successfully")

while True:
    print("""
1. Add Category
2. Add Expense
3. Update Expense
4. Delete Expense
5. Search Expense by Date
6. Category Analytics Report
7. Set Monthly Budget
8. Budget Alert
9. Exit
""")

    choice = input("Choose option: ")

    if choice == "1":
        add_category()
    elif choice == "2":
        add_expense()
    elif choice == "3":
        update_expense()
    elif choice == "4":
        delete_expense()
    elif choice == "5":
        search_by_date()
    elif choice == "6":
        category_report()
    elif choice == "7":
        set_budget()
    elif choice == "8":
        budget_alert()
    elif choice == "9":
        print("Exit") 
        break
    else:
        print("Invalid")




1. Add Category
2. Add Expense
3. Update Expense
4. Delete Expense
5. Search Expense by Date
6. Category Analytics Report
7. Set Monthly Budget
8. Budget Alert
9. Exit



Choose option:  8
Enter month to check:  2026-08


 No budget set for this month

1. Add Category
2. Add Expense
3. Update Expense
4. Delete Expense
5. Search Expense by Date
6. Category Analytics Report
7. Set Monthly Budget
8. Budget Alert
9. Exit

